In [1]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [2]:
from neo4j import GraphDatabase
import json, os

# --- Adjust these to your setup ---
URI      = "bolt://localhost:7687"
AUTH     = ("neo4j","password")   # your password here
REL_PATH = r"C:\Users\offic\AGENT\data\nlp\relations.json"


In [3]:
driver = GraphDatabase.driver(URI, auth=AUTH)
with driver.session() as session:
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (p:Paper)   REQUIRE p.id IS UNIQUE;")
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (m:Method)  REQUIRE m.name IS UNIQUE;")
    session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (d:Dataset) REQUIRE d.name IS UNIQUE;")
print("Constraints ensured.")


Constraints ensured.


In [4]:
# Load your extracted relations
with open(REL_PATH, encoding="utf-8") as f:
    triples = json.load(f)

def import_triples(triples):
    with driver.session() as session:
        for t in triples:
            params = {
                "pid":    t["paper_id"],
                "method": t["method"],
                "dataset":t["dataset"],
                "sent":   t.get("sentence",""),
                "metric": t.get("metric")
            }
            if params["metric"]:
                cypher = """
                MERGE (p:Paper {id:$pid})
                MERGE (m:Method {name:$method})
                MERGE (d:Dataset {name:$dataset})
                MERGE (p)-[:DESCRIBES]->(m)
                MERGE (m)-[r:USED_ON {metric:$metric}]->(d)
                SET r.sentence = $sent
                """
            else:
                cypher = """
                MERGE (p:Paper {id:$pid})
                MERGE (m:Method {name:$method})
                MERGE (d:Dataset {name:$dataset})
                MERGE (p)-[:DESCRIBES]->(m)
                MERGE (m)-[r:USED_ON]->(d)
                SET r.sentence = $sent
                """
            session.run(cypher, **params)

import_triples(triples)
print(f"Imported {len(triples)} relations.")


Imported 325838 relations.


In [5]:
# Run a small Cypher from Python
with driver.session() as session:
    result = session.run(
        "MATCH (m:Method)-[r:USED_ON]->(d:Dataset) "
        "RETURN m.name AS Method, d.name AS Dataset, r.metric AS Metric "
        "LIMIT 5"
    )
    for record in result:
        print(record)
driver.close()


<Record Method='ResNet' Dataset='ImageNet' Metric='1.0%'>
<Record Method='ViT' Dataset='ImageNet' Metric='95%'>
<Record Method='ResNet34' Dataset='ImageNet' Metric='1%'>
<Record Method='ViT-h' Dataset='ImageNet' Metric=None>
<Record Method='ViT-huge' Dataset='ImageNet' Metric=None>
